In [41]:
import pandas as pd
%matplotlib inline
import numpy as np
import turicreate as tc
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from spotlight.evaluation import precision_recall_score

In [4]:
#reading the data into a dataframe
data = pd.read_csv('/Users/armitasafavi/downloads/steam-200k.csv', names=['user_id','games','purchase_play','hours_played','none'])
data = data.drop('none', axis=1)

In [5]:
data.head(10)

,user_id,games,purchase_play,hours_played
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0
5,151603712,Spore,play,14.9
6,151603712,Fallout New Vegas,purchase,1.0
7,151603712,Fallout New Vegas,play,12.1
8,151603712,Left 4 Dead 2,purchase,1.0
9,151603712,Left 4 Dead 2,play,8.9


In [6]:
#take out the purchase duplicates
df = data.loc[data['purchase_play']== 'play']

In [7]:
df['hours_played'].describe()

count    70489.000000
mean        48.878063
std        229.335236
min          0.100000
25%          1.000000
50%          4.500000
75%         19.100000
max      11754.000000
Name: hours_played, dtype: float64

In [8]:
#define a rating scale from hours played by each user,  using below function
def rating(df):
    if (df['hours_played']< 1):
        return 1
    if (df['hours_played']> 1) and (df['hours_played']< 10):
        return 2
    if (df['hours_played']> 10) and (df['hours_played']< 20):
        return 3
    if (df['hours_played']> 20) and (df['hours_played']< 50):
        return 4
    if (df['hours_played']> 50) and (df['hours_played']< 100):
        return 5
    if (df['hours_played']> 100) and (df['hours_played']< 250):
        return 6
    if (df['hours_played']> 250) and (df['hours_played']< 500):
        return 7
    if (df['hours_played']> 500) and (df['hours_played']< 1000):
        return 8
    if (df['hours_played']> 1000) and (df['hours_played']< 1500):
        return 9
    else:
        return 10
    

In [9]:
#add ratings to the dataframe for each player and game
df['rating'] = df.apply(rating, axis=1)

/Users/armitasafavi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
#clean up the dataframe by dropping unnecessary columns
n_df = df.drop(columns=['purchase_play','hours_played'])
n_df.head()

,user_id,games,rating
1,151603712,The Elder Scrolls V Skyrim,7
3,151603712,Fallout 4,5
5,151603712,Spore,3
7,151603712,Fallout New Vegas,3
9,151603712,Left 4 Dead 2,2


In [12]:
#creating the user item matrix
df_matrix = pd.pivot_table(n_df, values='rating', index='user_id', columns='games')

In [13]:
df_matrix.head(10)

games,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,...,rFactor,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,the static speaks my name,theHunter,theHunter Primal
user_id,,,,,,,,,,,,,,,,,,,,,
5250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [16]:
#adding index and filling na for the user-item matrix
matrix=df_matrix.reset_index()
matrix1=matrix.fillna(0)

In [17]:
matrix1.head()

games,user_id,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,...,rFactor,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,the static speaks my name,theHunter,theHunter Primal
0,5250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,76767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,144736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,181212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Part 1 : Using turicreate library to define and compare 3 recommendation models


In [18]:
#create a table for input to the modeling  
d = df_matrix.reset_index()
d.index.names = ['rating']
data_norm = pd.melt(d, id_vars=['user_id'], value_name='rating').dropna()
print(data_norm.shape)
data_norm.head()

(70477, 3)


,user_id,games,rating
949,46055854,007 Legends,1.0
11533,11940338,0RBITALIS,1.0
13636,86055705,0RBITALIS,1.0
13921,93030550,0RBITALIS,1.0
23770,49893565,1... 2... 3... KICK IT! (Drop That Beat Like a...,2.0


In [19]:
# variables to define field names
# constant variables include:
t_data = tc.SFrame(data_norm) #creating dataframe readable for turicreate
user_id = 'user_id'
item_id = 'games'
users_to_recommend = list(matrix1[user_id])
n_rec = 10 # number of items to recommend
n_display = 20 # to print the head / first few rows in a defined dataset

In [20]:
#defining the model
def model(data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

popularity model

In [21]:

name = 'popularity'
target = 'rating'
pop_model = model(t_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 70477 observations with 11350 users and 3600 items.

Data prepared in: 0.345125s

70477 observations to process; with 3600 unique items.

recommendations finished on 1000/11350 queries. users per second: 32460.2

...

recommendations finished on 8000/11350 queries. users per second: 36185.7

recommendations finished on 9000/11350 queries. users per second: 36268.8

recommendations finished on 10000/11350 queries. users per second: 36340.6

recommendations finished on 11000/11350 queries. users per second: 37022.6

+---------+-------------------------------+-------+------+
| user_id |             games             | score | rank |
+---------+-------------------------------+-------+------+
|   5250  | Test Drive Ferrari Racing ... |  10.0 |  1   |
|   5250  |       East India Company      |  10.0 |  2   |
|   5250  |             Runers            |  10.0 |  3   |
|   5250  |          Dollar Dash          |  10.0 |  4   |
|   5250  |  Distant Star Revenant Fleet  |  10.0 |  5   |
|   5250  |        DinerTown Tycoon       |  10.0 |  6   |
|   5250  |        Bejeweled Deluxe       |  10.0 |  7   |
|   5250  |           Bad Hotel           |  10.0 |  8   |
|   5250  |            Asteria            |  10.0 |  9   |
|   5250  |          3D Mini Golf         |  10.0 |  10  |
|  76767  | Test Drive Ferrari Racing ... |  10.0 |  1   |
|  76767  |       East India Company      |  10.0 |  2   |
|  76767  |             Runers            |  10.0 |  3   |
|  76767  |          Dollar Dash          |  10.0 |  4  

As it appears , the model is recommending the same top 10 games 

cosine similarity model

In [36]:
name = 'cosine'
target = 'rating'
cosine_model = model(t_data, name, user_id, item_id, target, users_to_recommend, n_rec, 20)

Preparing data set.

Data has 70477 observations with 11350 users and 3600 items.

Data prepared in: 0.223251s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 8000/11350 queries. users per second: 28016.7

recommendations finished on 9000/11350 queries. users per second: 28440.2

recommendations finished on 10000/11350 queries. users per second: 28443.5

recommendations finished on 11000/11350 queries. users per second: 27769.8

+---------+-------------------------------+---------------------+------+
| user_id |             games             |        score        | rank |
+---------+-------------------------------+---------------------+------+
|   5250  |         Left 4 Dead 2         |  0.6339463094870249 |  1   |
|   5250  |     FTL Faster Than Light     |  0.6091278990109762 |  2   |
|   5250  |   The Elder Scrolls V Skyrim  |  0.5888828039169312 |  3   |
|   5250  |       XCOM Enemy Unknown      |  0.5690694252649943 |  4   |
|   5250  |         Borderlands 2         |  0.5247333844502767 |  5   |
|   5250  | The Witcher 2 Assassins of... |  0.5111501117547353 |  6   |
|   5250  |           Dishonored          | 0.48396604259808856 |  7   |
|   5250  |           Metro 2033          |  0.4633725682894389 |  8   |
|   5250  |          Borderlands          |  0.4522691567738851 |  9   |
|   5250  |          Half-Life 2          | 0.44557257493336994 |  10  |
|  76767  |         Left 4 Dead 2         |  0.3791

pearson similarity model

In [35]:
name = 'pearson'
target = 'rating'
pearson_model = model(t_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 70477 observations with 11350 users and 3600 items.

Data prepared in: 0.869796s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 8000/11350 queries. users per second: 17403.5

recommendations finished on 9000/11350 queries. users per second: 17922.4

recommendations finished on 10000/11350 queries. users per second: 17921.6

recommendations finished on 11000/11350 queries. users per second: 17910.1

+---------+-------------------------------+-------+------+
| user_id |             games             | score | rank |
+---------+-------------------------------+-------+------+
|   5250  |     The Curious Expedition    |  10.0 |  1   |
|   5250  | Test Drive Ferrari Racing ... |  10.0 |  2   |
|   5250  |             Runers            |  10.0 |  3   |
|   5250  |          Dollar Dash          |  10.0 |  4   |
|   5250  |  Distant Star Revenant Fleet  |  10.0 |  5   |
|   5250  |        DinerTown Tycoon       |  10.0 |  6   |
|   5250  |        Bejeweled Deluxe       |  10.0 |  7   |
|   5250  |           Bad Hotel           |  10.0 |  8   |
|   5250  |            Asteria            |  10.0 |  9   |
|   5250  |          3D Mini Golf         |  10.0 |  10  |
|  76767  |     The Curious Expedition    |  10.0 |  1   |
|  76767  | Test Drive Ferrari Racing ... |  10.0 |  2   |
|  76767  |             Runers            |  10.0 |  3   |
|  76767  |          Dollar Dash          |  10.0 |  4  

In [40]:

tc.recommender.util.compare_models(t_data, [pop_model, cosine_model, pearson_model], 
                                           metric='rmse')

PROGRESS: Evaluate model M0

Overall RMSE: 1.8218512680955279

Per User RMSE (best)
+-----------+------+-------+
|  user_id  | rmse | count |
+-----------+------+-------+
| 103966604 | 0.0  |   1   |
+-----------+------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+-----------+-------------------+-------+
|  user_id  |        rmse       | count |
+-----------+-------------------+-------+
| 130145396 | 8.040816326530612 |   1   |
+-----------+-------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+---------+------+-------+
|  games  | rmse | count |
+---------+------+-------+
| Burgers | 0.0  |   2   |
+---------+------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+-------------------------+------+-------+
|          games          | rmse | count |
+-------------------------+------+-------+
| Unreal II The Awakening | 4.5  |   4   |
+-------------------------+------+-------+
[1 rows x 3 columns]

PROGRESS: Evaluate model M1

Overall RMSE: 3.3326927505

[{'rmse_by_user': Columns:
  	user_id	int
  	rmse	float
  	count	int
  
  Rows: 11350
  
  Data:
  +-----------+--------------------+-------+
  |  user_id  |        rmse        | count |
  +-----------+--------------------+-------+
  | 284165884 | 1.6263168766783722 |   1   |
  | 238411403 | 2.626316876678372  |   1   |
  | 182593878 | 4.313597448784551  |   3   |
  | 207009485 | 1.2546740916047492 |   4   |
  | 217368291 | 3.373683123321628  |   1   |
  |  91035793 | 0.6263168766783722 |   1   |
  | 112084902 | 0.3736831233216278 |   1   |
  | 294715521 | 2.8048780487804876 |   1   |
  | 150974633 | 2.3854846253561144 |   2   |
  | 110881546 | 2.5844716740978964 |   22  |
  +-----------+--------------------+-------+
  [11350 rows x 3 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
  'rmse_by_item': Columns:
  	games	str
  	rmse	float
  	count	int
  
  Rows: 3600
  
  Data:
  +----------------

Part 2 : Building a user-item and item-item collaborative filtering

Item-item

In [25]:


# Start by normalizing the data by
#calculating the magnitude of all the user's ratings by taking the square root of the sum of the squares of all 
#the user's ratings.
data_items = matrix1.drop('user_id', 1)
# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_items).sum(axis=1))

# unitvector = (x / magnitude, y / magnitude, z / magnitude, ...)
data_items = data_items.divide(magnitude, axis='index')

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix
data_matrix = calculate_similarity(data_items)

# Lets get the top 10 similar games for Fallout 4
print(data_matrix.loc['Fallout 4'].nlargest(10))

games
Fallout 4                                 1.000000
The Witcher 3 Wild Hunt                   0.121778
Grand Theft Auto V                        0.108676
Putt-Putt Joins the Circus                0.108609
Putt-Putt Saves the Zoo                   0.108609
Putt-Putt Travels Through Time            0.108609
The Elder Scrolls V Skyrim                0.106004
Front Mission Evolved                     0.101082
Divinity Original Sin Enhanced Edition    0.097847
Aveyond 3-1 Lord of Twilight              0.096719
Name: Fallout 4, dtype: float64


In [62]:
df_matrix.index

Int64Index([     5250,     76767,     86540,    144736,    181212,    229911,
               298950,    381543,    547685,    554278,
            ...
            309228590, 309255941, 309262440, 309265377, 309404240, 309434439,
            309554670, 309626088, 309824202, 309903146],
           dtype='int64', name='user_id', length=11350)

In [99]:
user = 5250  # The id of the user for whom we want to generate recommendations
user_index = matrix1[matrix1.user_id == user].index.values.astype(int)[0] # Get the frame index

# Get the artists the user has liked.
known_user_likes = matrix1.loc[user_index]
known_user_likes_values = known_user_likes[known_user_likes >0]
known_user_likes_scores=known_user_likes_values.drop(known_user_likes_values.index[[0]]).index.values
# Users likes for all items as a sparse vector.
user_rating_vector = matrix1.loc[user_index]
user_rating_vector_score = user_rating_vector.drop(user_rating_vector.index[[0]])
#user_rating_vector= user_rating_vector.drop(user_rating_vector.index[[0]])

# Calculate the score.
score = data_matrix.dot(user_rating_vector_score).div(data_matrix.sum(axis=1))

# Remove the known likes from the recommendation.
score = score.drop(known_user_likes_scores)



# Print the known likes and the top 20 recommendations.
print (known_user_likes_values)
print( score.nlargest(10))

games
user_id                     5250.0
Alien Swarm                    2.0
Cities Skylines                6.0
Deus Ex Human Revolution       5.0
Dota 2                         1.0
Portal 2                       3.0
Team Fortress 2                1.0
Name: 0, dtype: float64
games
Choice of Robots                       0.268906
FINAL FANTASY XI                       0.249334
Song of the Myrne What Lies Beneath    0.235608
Chris Sawyer's Locomotion              0.141211
MX vs. ATV Supercross Encore           0.125719
Never Alone (Kisima Ingitchuna)        0.108152
Fork Parker's Holiday Profit Hike      0.099200
Bus-Simulator 2012                     0.097550
Medal of Honor Airborne                0.096402
GUN                                    0.095840
dtype: float64


In [124]:
users=n_df['user_id'].unique().tolist()

In [ ]:
#creating a recommendation for all users
recommend = {}
for user in users:
    
    user_index = matrix1[matrix1.user_id == user].index.values.astype(int)[0]
    

    known_user_likes = matrix1.loc[user_index]
    known_user_likes_values = known_user_likes[known_user_likes >0]
    known_user_likes_scores=known_user_likes_values.drop(known_user_likes_values.index[[0]]).index.values

    user_rating_vector = matrix1.loc[user_index]
    user_rating_vector_score = user_rating_vector.drop(user_rating_vector.index[[0]])



    score = data_matrix.dot(user_rating_vector_score).div(data_matrix.sum(axis=1))


    score = score.drop(known_user_likes_scores)
    recommend[user] = score.nlargest(10)


